In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train_df = pd.read_csv(r"C:\Users\dipes\Documents\dataset\titanic\train.csv")
test_df = pd.read_csv(r"C:\Users\dipes\Documents\dataset\titanic\test.csv")

In [3]:
data = [train_df, test_df]
for dataset in data:
    dataset['Relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['Relatives']> 0, 'Alone' ] = 'No'
    dataset.loc[dataset['Relatives'] == 0, 'Alone'] = 'Yes'
    #dataset['Alone'] = dataset['Alone'].astype(int)

In [4]:
train_df.Alone.value_counts()

Yes    537
No     354
Name: Alone, dtype: int64

In [5]:
import re
deck = {'A':'A', 'B':'B', 'C':'C', 'D':'D', 'E':'E', 'F':'F', 'G':'G', 'U':'U'}
data=[train_df, test_df]
for dataset in data:
    dataset['Cabin'] = dataset.Cabin.fillna('U0')
    dataset['Deck'] = dataset.Cabin.map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna('U')
    #dataset['Deck'] = dataset.Deck.astype(int)

In [6]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
 11  Relatives    891 non-null    int64  
 12  Alone        891 non-null    object 
 13  Deck         891 non-null    object 
dtypes: float64(2), int64(6), object(6)
memory usage: 97.6+ KB


In [8]:
data = [train_df, test_df]
for dataset in data:
    mean = train_df['Age'].mean()
    std = test_df['Age'].std()
    is_null = dataset['Age'].isnull().sum()
    rand_age = np.random.randint(mean - std, mean + std, size=is_null)
    age_slice = dataset['Age'].copy()
    age_slice[np.isnan(age_slice)]= rand_age
    dataset['Age'] = age_slice
    dataset['Age'] = dataset.Age.astype(int)

In [9]:
train_df.Age.isnull().sum()

0

In [10]:
dataset=[train_df, test_df]
for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  891 non-null    int64 
 1   Survived     891 non-null    int64 
 2   Pclass       891 non-null    int64 
 3   Name         891 non-null    object
 4   Sex          891 non-null    object
 5   Age          891 non-null    int32 
 6   SibSp        891 non-null    int64 
 7   Parch        891 non-null    int64 
 8   Ticket       891 non-null    object
 9   Fare         891 non-null    int32 
 10  Embarked     891 non-null    object
 11  Relatives    891 non-null    int64 
 12  Alone        891 non-null    object
 13  Deck         891 non-null    object
dtypes: int32(2), int64(6), object(6)
memory usage: 90.6+ KB


In [12]:
train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
train_df['Title'] = train_df['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Lady', 'Countess', 'Don', 'Capt', 'Jonkheer', 'Sir'], 'Rare')
train_df['Title'] = train_df['Title'].replace(['Mlle', 'Ms'], 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

In [13]:
train_df['Title'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: Title, dtype: int64

In [14]:
test_df['Title'] = test_df.Name.str.extract(' ([A-Za-z]+)\. ', expand=False)
test_df['Title']= test_df['Title'].replace(['Col', 'Rev', 'Dona', 'Dr'], 'Rare')
test_df['Title']= test_df['Title'].replace('Ms', 'Miss')
test_df['Title'].value_counts()

Mr        240
Miss       79
Mrs        72
Master     21
Rare        6
Name: Title, dtype: int64

In [15]:
train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

In [18]:
data = [train_df, test_df]
for dataset in data:
    dataset['Age_Class']= dataset['Age']* dataset['Pclass']

In [19]:
for dataset in data:
    dataset['Fare_Per_Person'] = dataset['Fare']/(dataset['Relatives']+1)
    dataset['Fare_Per_Person'] = dataset['Fare_Per_Person'].astype(int)

In [20]:
data = [train_df, test_df]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 7
    
    dataset['Age'] = dataset['Age'].astype(str)
    dataset.loc[ dataset['Age'] == '0', 'Age'] = "Children"
    dataset.loc[ dataset['Age'] == '1', 'Age'] = "Teens"
    dataset.loc[ dataset['Age'] == '2', 'Age'] = "Youngsters"
    dataset.loc[ dataset['Age'] == '3', 'Age'] = "Young Adults"
    dataset.loc[ dataset['Age'] == '4', 'Age'] = "Adults"
    dataset.loc[ dataset['Age'] == '5', 'Age'] = "Middle Age"
    dataset.loc[ dataset['Age'] == '6', 'Age'] = "Senior"
    dataset.loc[ dataset['Age'] == '7', 'Age'] = "Retired"

# let's see how it's distributed 
train_df['Age'].value_counts()

Adults          174
Senior          147
Middle Age      146
Young Adults    135
Youngsters      117
Teens            97
Children         68
Retired           7
Name: Age, dtype: int64

In [21]:
data = [train_df, test_df]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    dataset['Fare'] = dataset['Fare'].astype(str)
    dataset.loc[ dataset['Fare'] == '0', 'Fare'] = "Extremely Low"
    dataset.loc[ dataset['Fare'] == '1', 'Fare'] = "Very Low"
    dataset.loc[ dataset['Fare'] == '2', 'Fare'] = "Low"
    dataset.loc[ dataset['Fare'] == '3', 'Fare'] = "High"
    dataset.loc[ dataset['Fare'] == '4', 'Fare'] = "Very High"
    dataset.loc[ dataset['Fare'] == '5', 'Fare'] = "Extremely High"

In [22]:
data = [train_df, test_df]

for dataset in data:
    dataset['Pclass'] = dataset['Pclass'].astype(str)
    dataset.loc[ dataset['Pclass'] == '1', 'Pclass'] = "Class1"
    dataset.loc[ dataset['Pclass'] == '2', 'Pclass'] = "Class2"
    dataset.loc[ dataset['Pclass'] == '3', 'Pclass'] = "Class3"

In [23]:
train_df = train_df.drop(['PassengerId'], axis=1)

In [24]:
train_df = train_df.drop(['Ticket'], axis=1)

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Survived         891 non-null    int64 
 1   Pclass           891 non-null    object
 2   Sex              891 non-null    object
 3   Age              891 non-null    object
 4   SibSp            891 non-null    int64 
 5   Parch            891 non-null    int64 
 6   Fare             891 non-null    object
 7   Embarked         891 non-null    object
 8   Relatives        891 non-null    int64 
 9   Alone            891 non-null    object
 10  Deck             891 non-null    object
 11  Title            891 non-null    object
 12  Age_Class        891 non-null    int64 
 13  Fare_Per_Person  891 non-null    int32 
dtypes: int32(1), int64(5), object(8)
memory usage: 94.1+ KB


In [26]:
test_df = test_df.drop(['Ticket'], axis=1)

In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PassengerId      418 non-null    int64 
 1   Pclass           418 non-null    object
 2   Sex              418 non-null    object
 3   Age              418 non-null    object
 4   SibSp            418 non-null    int64 
 5   Parch            418 non-null    int64 
 6   Fare             418 non-null    object
 7   Embarked         418 non-null    object
 8   Relatives        418 non-null    int64 
 9   Alone            418 non-null    object
 10  Deck             418 non-null    object
 11  Title            418 non-null    object
 12  Age_Class        418 non-null    int64 
 13  Fare_Per_Person  418 non-null    int32 
dtypes: int32(1), int64(5), object(8)
memory usage: 44.2+ KB


In [28]:
train_numerical_features = list(train_df.select_dtypes(include=['int64', 'float64', 'int32']).columns)
train_numerical_features

['Survived', 'SibSp', 'Parch', 'Relatives', 'Age_Class', 'Fare_Per_Person']

In [29]:
del train_numerical_features[0]
train_numerical_features

['SibSp', 'Parch', 'Relatives', 'Age_Class', 'Fare_Per_Person']

In [30]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer
ss_scaler = StandardScaler()
train_df_ss = pd.DataFrame(data = train_df)
train_df_ss[train_numerical_features] = ss_scaler.fit_transform(train_df_ss[train_numerical_features])

In [31]:
test_numerical_features = list(test_df.select_dtypes(include=['int64', 'float64', 'int32']).columns)
test_numerical_features

['PassengerId', 'SibSp', 'Parch', 'Relatives', 'Age_Class', 'Fare_Per_Person']

In [32]:
train_df_ss.shape

(891, 14)

In [33]:
del test_numerical_features[0]
test_numerical_features

['SibSp', 'Parch', 'Relatives', 'Age_Class', 'Fare_Per_Person']

In [34]:
test_ss_scaler = StandardScaler()
test_df_ss = pd.DataFrame(data = test_df)
test_df_ss[test_numerical_features] = test_ss_scaler.fit_transform(test_df_ss[test_numerical_features])

In [35]:
test_df_ss.shape

(418, 14)

In [36]:
encode_col_list = list(train_df.select_dtypes(include=['object']).columns)
for i in encode_col_list:
    train_df_ss = pd.concat([train_df_ss,pd.get_dummies(train_df_ss[i], prefix=i)],axis=1)
    train_df_ss.drop(i, axis = 1, inplace=True)

In [42]:
train_df_ss.to_csv(r"C:\Users\dipes\Documents\dataset\titanic\trainset.csv", index=False)

In [37]:
train_df_ss

,Survived,SibSp,Parch,Relatives,Age_Class,Fare_Per_Person,Pclass_Class1,Pclass_Class2,Pclass_Class3,Sex_female,...,Deck_D,Deck_E,Deck_F,Deck_G,Deck_U,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,0,0.432793,-0.473674,0.059160,0.059429,-0.459218,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
1,1,0.432793,-0.473674,0.059160,-0.774447,0.434090,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,-0.474545,-0.473674,-0.560975,0.416804,-0.347554,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
3,1,0.432793,-0.473674,0.059160,-0.863791,0.182847,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,-0.474545,-0.473674,-0.560975,1.220899,-0.319638,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,-0.474545,-0.473674,-0.560975,-0.297947,-0.180059,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
887,1,-0.474545,-0.473674,-0.560975,-1.340292,0.294511,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
888,0,0.432793,2.008933,1.299429,0.416804,-0.403386,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
889,1,-0.474545,-0.473674,-0.560975,-1.131823,0.294511,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [38]:
test_encode_col_list = list(test_df.select_dtypes(include=['object']).columns)
for i in test_encode_col_list:
    test_df_ss = pd.concat([test_df_ss,pd.get_dummies(test_df_ss[i], prefix=i)],axis=1)
    test_df_ss.drop(i, axis = 1, inplace=True)

In [39]:
test_df_ss.shape

(418, 43)

In [40]:
test_df_ss.to_csv(r"C:\Users\dipes\Documents\dataset\titanic\testset.csv", index=False)

In [40]:
X_train = train_df_ss.drop("Survived", axis=1)
Y_train = train_df_ss["Survived"]
X_test  = test_df_ss.drop("PassengerId", axis=1).copy()

In [41]:
X_train.shape, X_test.shape, Y_train.shape

((891, 42), (418, 42), (891,))

In [42]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(X_train,Y_train,test_size=0.2,random_state=42)

In [43]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(criterion='gini', max_depth=8, min_samples_leaf=2, min_samples_split=4, n_estimators=500, n_jobs=1)
rfc.fit(trainx, trainy)
y_pred = rfc.predict(testx)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(testy, y_pred)

0.8268156424581006

In [45]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(trainx, trainy)
pred = logreg.predict(testx)
accuracy_score(testy, pred)

0.8156424581005587

In [46]:
from sklearn import svm
rbf_svc = svm.SVC(kernel='rbf')
rbf_svc.fit(trainx, trainy)
preds=rbf_svc.predict(testx)
accuracy_score(testy, preds)

0.8156424581005587

In [47]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=3, min_samples_split=5)
dtc.fit(trainx, trainy)
preds= dtc.predict(testx)
accuracy_score(testy, preds)

0.8324022346368715

In [54]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(trainx, trainy)
knn.score(testx, testy)

0.8324022346368715